In [1]:
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# SUGERENCIA: Considera dividir esta celda en múltiples celdas más pequeñas
# Código mejorado - 2025-08-11
import time
import os
from pathlib import Path
from datetime import datetime
import json
import matplotlib.pyplot as plt
from IPython.display import clear_output
import pandas as pd

class MIPProjectMonitor:
    def __init__(self, project_path):
        self.project_path = Path(project_path)
        self.monitoring_data = []
        self.last_check = {}
    
    def check_project_status(self):
        """Verifica el estado actual del proyecto"""
        notebooks = list(self.project_path.glob("*.ipynb"))
        
        status = {
            'timestamp': datetime.now().isoformat(),
            'total_notebooks': len(notebooks),
            'total_size_mb': sum(nb.stat().st_size for nb in notebooks) / (1024*1024),
            'modified_recently': 0,
            'notebooks_status': {}
        }
        
        # Verificar modificaciones recientes (últimas 24 horas)
        current_time = time.time()
        for nb in notebooks:
            mod_time = nb.stat().st_mtime
            is_recent = (current_time - mod_time) < 86400  # 24 horas
            
            if is_recent:
                status['modified_recently'] += 1
            
            status['notebooks_status'][nb.name] = {
                'size_kb': nb.stat().st_size / 1024,
                'modified': datetime.fromtimestamp(mod_time).isoformat(),
                'modified_recently': is_recent
            }
        
        return status
    
    def detect_issues(self, current_status):
        """Detecta posibles problemas en el proyecto"""
        issues = []
        
        # Issue 1: Notebooks muy grandes
        for nb_name, nb_data in current_status['notebooks_status'].items():
            if nb_data['size_kb'] > 5000:  # > 5MB
                issues.append(f"⚠️ {nb_name} es muy grande ({nb_data['size_kb']:.1f} KB)")
        
        # Issue 2: Mucha actividad reciente (posible trabajo intensivo)
        if current_status['modified_recently'] > 3:
            issues.append(f"🔥 Alta actividad: {current_status['modified_recently']} notebooks modificados recientemente")
        
        # Issue 3: Comparar con check anterior
        if self.last_check:
            size_growth = current_status['total_size_mb'] - self.last_check.get('total_size_mb', 0)
            if size_growth > 10:  # Crecimiento > 10MB
                issues.append(f"📈 Proyecto creció significativamente: +{size_growth:.1f} MB")
        
        return issues
    
    def generate_recommendations(self, current_status, issues):
        """Genera recomendaciones basadas en el estado"""
        recommendations = []
        
        if current_status['total_size_mb'] > 50:
            recommendations.append("💾 Considera limpiar outputs de notebooks para reducir tamaño")
        
        if current_status['modified_recently'] == 0:
            recommendations.append("😴 No hay actividad reciente. ¿Proyecto pausado?")
        
        if len(issues) > 0:
            recommendations.append("🔍 Ejecuta auditoría completa para análisis detallado")
        
        if current_status['total_notebooks'] > 10:
            recommendations.append("📁 Considera organizar notebooks en subdirectorios")
        
        return recommendations
    
    def display_dashboard(self, current_status, issues, recommendations):
        """Muestra dashboard en tiempo real"""
        clear_output(wait=True)
        
        print("🌡️ MONITOR CONTINUO - PROYECTO MIP QUILLOTA")
        print("="*60)
        print(f"🕒 Último check: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print()
        
        # Estado general
        print("📊 ESTADO GENERAL:")
        print(f"  📁 Total notebooks: {current_status['total_notebooks']}")
        print(f"  💾 Tamaño total: {current_status['total_size_mb']:.1f} MB")
        print(f"  🔥 Modificados hoy: {current_status['modified_recently']}")
        print()
        
        # Issues encontrados
        if issues:
            print("⚠️ ISSUES DETECTADOS:")
            for issue in issues:
                print(f"  {issue}")
            print()
        else:
            print("✅ No se detectaron issues\n")
        
        # Recomendaciones
        if recommendations:
            print("💡 RECOMENDACIONES:")
            for rec in recommendations:
                print(f"  {rec}")
            print()
        
        # Top 5 notebooks más grandes
        notebooks_by_size = sorted(
            current_status['notebooks_status'].items(),
            key=lambda x: x[1]['size_kb'],
            reverse=True
        )[:5]
        
        print("📋 TOP 5 NOTEBOOKS MÁS GRANDES:")
        for nb_name, nb_data in notebooks_by_size:
            recent_icon = "🔥" if nb_data['modified_recently'] else "  "
            print(f"  {recent_icon} {nb_name[:30]:<30} {nb_data['size_kb']:>8.1f} KB")
        
        print("\n" + "="*60)
    
    def save_monitoring_data(self):
        """Guarda datos de monitoreo"""
        if self.monitoring_data:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            monitor_path = self.project_path / f"monitoring_data_{timestamp}.json"
            
            with open(monitor_path, 'w', encoding='utf-8') as f:
                json.dump(self.monitoring_data, f, indent=2)
            
            print(f"💾 Datos de monitoreo guardados en: {monitor_path}")
    
    def plot_project_evolution(self):
        """Grafica la evolución del proyecto"""
        if len(self.monitoring_data) < 2:
            print("📊 Necesitas más datos para mostrar evolución")
            return
        
        df = pd.DataFrame(self.monitoring_data)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Evolución del tamaño
        ax1.plot(df['timestamp'], df['total_size_mb'], marker='o', linewidth=2)
        ax1.set_title('📈 Evolución del Tamaño del Proyecto')
        ax1.set_ylabel('Tamaño (MB)')
        ax1.grid(True, alpha=0.3)
        
        # Actividad diaria
        ax2.plot(df['timestamp'], df['modified_recently'], marker='s', linewidth=2, color='orange')
        ax2.set_title('🔥 Actividad Diaria (Notebooks Modificados)')
        ax2.set_ylabel('Notebooks Modificados')
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def start_monitoring(self, interval_minutes=5, duration_hours=2):
        """Inicia monitoreo continuo"""
        print(f"🚀 Iniciando monitoreo por {duration_hours} horas...")
        print(f"🔄 Checks cada {interval_minutes} minutos")
        print("🛑 Presiona Ctrl+C para detener\n")
        
        start_time = time.time()
        end_time = start_time + (duration_hours * 3600)
        
        try:
            while time.time() < end_time:
                # Realizar check
                current_status = self.check_project_status()
                issues = self.detect_issues(current_status)
                recommendations = self.generate_recommendations(current_status, issues)
                
                # Mostrar dashboard
                self.display_dashboard(current_status, issues, recommendations)
                
                # Guardar datos
                self.monitoring_data.append(current_status)
                self.last_check = current_status
                
                # Mostrar progreso
                elapsed = time.time() - start_time
                remaining = end_time - time.time()
                print(f"⏱️  Tiempo transcurrido: {elapsed/60:.1f} min | Restante: {remaining/60:.1f} min")
                
                # Esperar siguiente check
                time.sleep(interval_minutes * 60)
        
        except KeyboardInterrupt:
            print("\n🛑 Monitoreo detenido por usuario")
        
        # Guardar datos finales
        self.save_monitoring_data()
        
        # Mostrar gráfico de evolución
        if len(self.monitoring_data) > 1:
            self.plot_project_evolution()
        
        print("✅ Monitoreo completado")

# Monitor de uso inmediato
monitor = MIPProjectMonitor(r"C:\Users\Alicia_Piero\Documents\Repo_AIEP\MIP_QUILLOTA\Proyecto_METGO_3D")

# Para un check rápido:
def quick_check():
    """Realiza un check rápido del proyecto"""
    status = monitor.check_project_status()
    issues = monitor.detect_issues(status)
    recommendations = monitor.generate_recommendations(status, issues)
    monitor.display_dashboard(status, issues, recommendations)

# Ejecutar check rápido
quick_check()

# Para monitoreo continuo (descomenta la línea siguiente):
# monitor.start_monitoring(interval_minutes=2, duration_hours=1)

🌡️ MONITOR CONTINUO - PROYECTO MIP QUILLOTA
🕒 Último check: 2025-08-11 12:55:50

📊 ESTADO GENERAL:
  📁 Total notebooks: 12
  💾 Tamaño total: 7.4 MB
  🔥 Modificados hoy: 10

⚠️ ISSUES DETECTADOS:
  🔥 Alta actividad: 10 notebooks modificados recientemente

💡 RECOMENDACIONES:
  🔍 Ejecuta auditoría completa para análisis detallado
  📁 Considera organizar notebooks en subdirectorios

📋 TOP 5 NOTEBOOKS MÁS GRANDES:
  🔥 04_Visualizaciones.ipynb         3303.2 KB
  🔥 03_Analisis_Meteorologico.ipyn   1719.2 KB
     Sistema de Pronóstico Meteorol    885.5 KB
  🔥 05_Modelos_ML.ipynb               416.7 KB
  🔥 00_Sistema_Principal_MIP_Quill    311.9 KB

